In [12]:
!pip3 install python-statemachine
%run helpers.ipynb
%run math_utils.ipynb

In [3]:
from statemachine import StateMachine, State
import numpy as np

In [1]:
class ADCS(StateMachine):
    """
    This class functions as the ADCS Controller.
    
    inputs = []
    outputs = []
    states = []
    
    transitions
    -----------
    
    """
    
    off = State(initial=True)
    detumbling = State()
    pointing = State()
    error = State()
    
    on_initialization = off.to(detumbling)
    to_pointing = detumbling.to(pointing)
    continuous_pointing = pointing.to(pointing)
    
    to_error = pointing.to(error)

    I_error = 0
    D_error = 0
    prev_error = np.zeros(3)
    
    def __init__(self):
        self.moment = None
        self.pointing_vec = [0, 0, 0]
        super(ADCS, self).__init__()
        
    def b_dot(self, mag_field_vector, cang):
        detb = np.sqrt(np.dot(mag_field_vector, mag_field_vector))
        m = ((-1) / detb) * np.cross(mag_field_vector, cang)    
        return 2 * m

    def get_attitude_err(self, q_target, q_curr):
        q_err = quaternion_multiply(q_curr, quaternion_inverse(q_target))
        roll, pitch, yaw = quaternion_to_euler(q_err)

        # For some reason switching these when correcting the error makes the PID controller work
        return np.array([roll, pitch, yaw])

    def get_attitude_rate_err(self, w_desired, w_estimated, attitude_err):
        delta_w = w_estimated - w_desired
        attitude_rate_err = delta_w # -cross(w_estimated, attitude_err) 
        
        mag = np.linalg.norm(attitude_rate_err)
        if mag < np.finfo(np.float64).eps:
            return np.zeros(attitude_rate_err.shape)
            
        return attitude_rate_err

    def get_pointing_torque(self, q_target, q_curr, w_desired, w_estimated):
        attitude_err = self.get_attitude_err(q_target, q_curr)
        self.I_error += attitude_err
        self.D_error = attitude_err - self.prev_error
        self.prev_error = attitude_err
        attitude_rate_err = self.get_attitude_rate_err(w_desired, w_estimated, attitude_err)
        u = attitude_err * -8e-6 + attitude_rate_err * -1e-1
        # u = attitude_err * -5e-6 + attitude_rate_err * -5e-2
        J = inertia(6, 0.34, 0.1, 0.1)
        return np.matmul(J, u)

    def get_pointing_moment(self, magnetic_field, q_target, q_curr, w_desired, w_estimated):
        torque = self.get_pointing_torque(q_target, q_curr, w_desired, w_estimated)
        torque_perp = torque - (np.dot(torque, magnetic_field)/np.dot(magnetic_field, magnetic_field)) * magnetic_field
        moment = (1/np.dot(magnetic_field, magnetic_field)) * np.cross(magnetic_field, torque_perp)
        return moment
    
    def detumble(self, mag_field, cang):
        # slow down rotation
        self.moment = self.b_dot(mag_field, cang)
        
    def point(self, mag_field, q_target, q_curr, w_desired, w_estimated):
        # update pointing vector
        self.moment = self.get_pointing_moment(mag_field, q_target, q_curr, w_desired, w_estimated)
    
    def get_moment(self):
        # Limit maximum moment
        # m_max = 0.181
        # m_mag = np.linalg.norm(self.moment)
        # if m_mag > m_max:
        #     self.moment = (m_max/m_mag) * self.moment
        return self.moment

class SafetyMonitor(StateMachine):
    
    safe = State(initial=True)
    error = State()
    
    to_error = safe.to(error, cond="moment_error")
    
    def moment_error(moment):
        return moment > 0.12
    
    def get_state(self):
        return self.current_state.id

NameError: name 'StateMachine' is not defined

In [7]:
# Given vector t and vector B
t = np.array([1, 2, 3])  # Example vector t
B = np.array([4, 5, 6])  # Example vector B

# Calculate magnitude of vector B
magnitude_B = np.linalg.norm(B)

# Calculate cross product of m and B
m_cross_B = t

# Calculate magnitude of cross product
magnitude_m_cross_B = np.linalg.norm(m_cross_B)

# Calculate sin(theta)
sin_theta = magnitude_m_cross_B / (magnitude_B)

# Calculate angle theta in radians
theta_radians = np.arcsin(sin_theta)

# Calculate angle theta in degrees
theta_degrees = np.degrees(theta_radians)

# Calculate m
m = m_cross_B / (magnitude_B * sin_theta)

print("Vector m:", m)
print(np.cross(m,B))
print("Angle theta (in degrees):", theta_degrees)

Vector m: [0.26726124 0.53452248 0.80178373]
[-0.80178373  1.60356745 -0.80178373]
Angle theta (in degrees): 25.23940182067891
